<h1>Dependencies</h1>

In [2]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredWordDocumentLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
import pypdf

<h1>Embed Class Slides and Textbooks</h1>

In [ ]:
# Load documents
loader = DirectoryLoader('../docs', glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma.from_documents(persist_directory='../chroma', documents=splits, embedding=embeddings)

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong poi

<h1>Initialize Llama LLM locally</h1>

In [5]:
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma(persist_directory='../chroma', embedding_function=embeddings)
# Initialize LLama 3.2
llm = Ollama(model="llama3.2", base_url="http://localhost:11434")



<h1>Generate prompt with GUI using gradio</h1>

In [ ]:
import gradio as gr
import random

def answer(message: str, history: list[str]):
    prompt = "\n".join([f"User: {item[0]}\nBot: {item[1]}" for item in history]) + f"\nUser: {message}\nBot:"
    similar_docs = vectorstore.similarity_search(prompt)
    context = "\n".join([doc.page_content for doc in similar_docs])
    response = llm(f"Context: {context}\nQuestion: {prompt}\nAnswer:")
    history.append([message, response])

    return response


chatbot = gr.ChatInterface(
    fn=answer,
    title="Group 5 Quizbot",
    description="Enter the following prompt to be quizzed: Generate a multiple choice, true/false, or short answer network security question and provide feedback to my answer."
)

chatbot.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_114370/2628795082.py:8: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(f"Context: {context}\nQuestion: {prompt}\nAnswer:")
